#### 1. 3개 이상 문장 유사도 분석 모델 

- 토큰화에서 형태소 분석 시 사용할 모델로 Okt()혹은 Kkma() 중에서 성능이 더 잘 나오는 모델로 선택할 필요성 존재 
https://konlpy.org/ko/v0.6.0/morph/
- 벡터화는 TfidfVectorizer 사용 
- 3개 이상의 문장을 input 받을 때는 cosine_similarity_matrix 사용 
- 2개의 문장을 비교할 때는 numpy를 사용하여 cosine similarity 공식 적용 


##### 1.1 코사인 유사도 분석 

In [57]:
# 문장이 여러개 입력 받는 경우 
from konlpy.tag import Kkma 
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_matrix(x_data):

    okt = Okt()
    for i, document in enumerate(x_data):
        nouns = okt.morphs(document)    
        x_data[i] = ' '.join(nouns)

    # print(x_data)

    vect = TfidfVectorizer()
    x_data = vect.fit_transform(x_data)
    cosine_similarity_matrix = (x_data * x_data.T)

    # print(cosine_similarity_matrix.shape)
    # print(cosine_similarity_matrix)

    return cosine_similarity_matrix.toarray()

x_data = np.array(['영희가 사랑하는 강아지 백구를 산책시키고 있다.',
        '철수가 사랑하는 소 누렁이를 운동시키고 있다.'])
        # '영희와 철수는 소와 강아지를 산책 및 운동시키고 있다.'])
        
print(cosine_similarity_matrix(x_data))

[[1.         0.22576485]
 [0.22576485 1.        ]]


#### 2. 1대 1 유사도 분석 모델 
##### 2.1 코사인 유사도 분석  

In [58]:
# 문장이 2개 입력되는 경우
from konlpy.tag import Kkma     # 꼬꼬마 사용 
from konlpy.tag import Okt
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 

def cos_similarity(x_data):
        
        kkma = Kkma()
                
        doc_nouns_list = []

        for data in x_data:
                nouns = kkma.morphs(data)
                doc_nouns = ''

                for noun in nouns:
                        doc_nouns+= noun + ' '

                doc_nouns_list.append(doc_nouns)

        vect = TfidfVectorizer()
        x_data = vect.fit_transform(doc_nouns_list)

        feature_vect_dense = x_data.todense()      # 벡터화

        v1 = np.array(feature_vect_dense[0]).reshape(-1,)
        v2 = np.array(feature_vect_dense[1]).reshape(-1,)

        dot_product = np.dot(v1, v2)
        l2_norm = (np.linalg.norm(v1)*np.linalg.norm(v2))
        return dot_product / l2_norm

x_data = ['영희가 사랑하는 강아지 백구를 산책시키고 있다.', '철수가 사랑하는 소 누렁이를 운동시키고 있다.']
print(cos_similarity(x_data))

0.2257648460026161


```
같은 문장으로 유사도 분석을 해 봤을 때 
1) Okt() + cosine_similarity_matrix : 유사도 0.30412574
2) Okt() + numpy : 유사도 0.3680232087561149
3) Kkma() + cosine_similarity_matrix : 0.22576485
4) Kkma() + numpy : 유사도 0.2257648460026161
```

##### 2.2 레벤슈타인 거리 

In [65]:
#8. Levenschtein distance
import numpy as np
def leven(text1, text2):
    len1 = len(text1) + 1
    len2 = len(text2) + 1
    sim_array = np.zeros((len1, len2))
    sim_array[:,0] = np.linspace(0, len1-1, len1)  # sim_array의 0번째 컬럼의 모든 행에 대해 0부터 len1-1까지 len1등분해서 채우기 
    sim_array[0,:] = np.linspace(0, len2-1, len2)
    for i in range(1,len1):
        for j in range(1,len2):
            add_char = sim_array[i-1,j] + 1
            sub_char = sim_array[i,j-1] + 1
            if text1[i-1] == text2[j-1]:
                mod_char = sim_array[i-1,j-1]
            else:
                mod_char = sim_array[i-1,j-1] + 1
            sim_array[i,j] = min([add_char, sub_char, mod_char])
    return sim_array[-1,-1]
print(leven('데이터마이닝','데이타마닝'))
print(leven('영희가 사랑하는 강아지 백구를 산책시키고 있다.', '철수가 사랑하는 소 누렁이를 운동시키고 있다.'))

2.0
10.0


In [ ]:
def calc_distance(a, b):
    if a==b: return 0
    a_len=len(a)
    b_len=len(b)

### 1.3 N-gram 유사도 분석 

In [23]:
# n-gram 어휘 벡터 생성 

def ngram(s, num):
    res = []
    slen = len(s)-num+1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

In [63]:
# n-gram 유사도 분석
def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)

    r = []
    cnt = 0
    for i in a :
        for j in b:
            if i==j:
                cnt += 1
                r.append(i)
    return cnt/len(a)

a = "오늘 강남에서 맛있는 스파게티를 먹었다."
b = "강남에서 먹었던 오늘의 스파게티는 맛있었다."

word2 = diff_ngram(a, b, 2)
print("2-gram:", word2)

word3 = diff_ngram(a, b, 3)
print("3-gram:", word3)

word4 = diff_ngram(a, b, 4)
print("4-gram:", word4)

2-gram: 0.7619047619047619
3-gram: 0.45
4-gram: 0.21052631578947367


### 결론 
- 코사인 유사도 분석 모델은 유사도가 백분위 숫자로 출력됨
- 레벤슈타인 거리는 유사할 수록 낮은 수치가, 유사하지 않을수록 높은 수치로 나타난다. 0~ 범위의 숫자로 값이 출력되며 문장이 길어질수록 점점 수치가 커질 수 있다.
- n-gram 유사도의 경우 유사도가 백분위 숫자로 출력된다. 또한 n이 커질수록 유사도가 수치가 점점 낮아진다. => 적절한 n을 선택하는 것이 관건 